In [672]:
import warnings
warnings.filterwarnings('ignore')
#!7z.exe e dota2.zip


In [238]:
%cd dota2\dota2

[WinError 3] Системе не удается найти указанный путь: 'dota2\\dota2'
C:\Grisha\МФТИ\МашинноеОсновыЮдин\HW2\env\dota2\dota2


In [7]:
#!7z.exe e features.csv.zip


7-Zip 19.00 (x64) : Copyright (c) 1999-2018 Igor Pavlov : 2019-02-21

Scanning the drive for archives:
1 file, 11608616 bytes (12 MiB)

Extracting archive: features.csv.zip
--
Path = features.csv.zip
Type = zip
Physical Size = 11608616

Everything is Ok

Size:       33074264
Compressed: 11608616


In [676]:
import pandas as pd
import numpy as np

data = pd.read_csv('features.csv')

In [677]:
data.head(3)

,match_id,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
0,0,1430198770,7,11,5,2098,1489,20,0,0,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1,1430220345,0,42,4,1188,1033,9,0,1,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,2,1430227081,7,33,4,1319,1270,22,0,0,...,4,3,1,13.0,2130,0,0,1830,0,63


In [678]:
data.columns[:10]

Index(['match_id', 'start_time', 'lobby_type', 'r1_hero', 'r1_level', 'r1_xp',
       'r1_gold', 'r1_lh', 'r1_kills', 'r1_deaths'],
      dtype='object')

In [679]:
#Проверяем наличие nan в данных
data.isnull().sum().sum()

193087

In [680]:
#col_nan - имена колонок с пропущенными значениями
col_ind_nan = np.arange(data.columns.shape[0])[data.isnull().sum() > 0]
col_nan = data.columns[col_ind_nan]

In [681]:
data[col_nan].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97230 entries, 0 to 97229
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   first_blood_time             77677 non-null  float64
 1   first_blood_team             77677 non-null  float64
 2   first_blood_player1          77677 non-null  float64
 3   first_blood_player2          53243 non-null  float64
 4   radiant_bottle_time          81539 non-null  float64
 5   radiant_courier_time         96538 non-null  float64
 6   radiant_flying_courier_time  69751 non-null  float64
 7   radiant_first_ward_time      95394 non-null  float64
 8   dire_bottle_time             81087 non-null  float64
 9   dire_courier_time            96554 non-null  float64
 10  dire_flying_courier_time     71132 non-null  float64
 11  dire_first_ward_time         95404 non-null  float64
dtypes: float64(12)
memory usage: 8.9 MB


In [682]:
#Признак - время возникновения события, если событие не возникает то время его появления -1.
event_time_features = ["first_blood_time", "radiant_bottle_time", "radiant_courier_time", 
                       "radiant_flying_courier_time", "radiant_first_ward_time", "dire_bottle_time", 
                      "dire_courier_time", "dire_flying_courier_time", "dire_first_ward_time"]

for feat in event_time_features:
    data[feat] = data[feat].fillna(-1)

    
#Признаки first blood player. значение признака NAN в том случае, если первая кровь не произошла.
#по этому оба признака будут равны номеру не существующего игрока 0.
data["first_blood_player1"] = data["first_blood_player1"].fillna(0)
data["first_blood_player2"] = data["first_blood_player2"].fillna(0)

#Признак first_blood_team равен -1 если ни одна из команд не совершила первую кровь
data["first_blood_team"] = data["first_blood_team"].fillna(-1)

data.isnull().sum().sum()

0

In [683]:
#Выделяем целевую переменную, а так же признаки
target = ['radiant_win']
features = list(filter(lambda x: x != target[0], data.columns))

#Пока не будем учитывать те признаки, которых нет в тестовых данных в целях формирования baseline
#Так как опытным путем проверенно, что эти признаки почти однозначно определяют команду победителя,
#то обучая модель на этих признаках, нам придется их искуственно генерировать для тестовых данных
#тем самым мы искуственно сгенерируем таргет признак для тестовых данных.
added_features = ["duration",
                 "tower_status_radiant",
                 "tower_status_dire",
                 "barracks_status_radiant",
                 "barracks_status_dire"]

for feat in added_features: features.remove(feat)
#так же удалим признаки, которые не несут информации для предсказания
features.remove("match_id")
features.remove("start_time")

features = np.array(features)

In [684]:
#Так как соревнования уже не существует, будем использовать в качестве тестовой выборки 20% от текущей
from sklearn.model_selection import train_test_split
X = data[features].to_numpy()
y = data[target].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [12]:
!pip install xgboost

In [227]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score, precision_score, recall_score

In [198]:
# Обучение
auc_scorer = make_scorer(roc_auc_score)
model = XGBClassifier(eval_metric='mlogloss',  
                      use_label_encoder=False, 
                      n_jobs=-1)

#result = cross_val_score(model, X, y, scoring=auc_scorer, cv=5)
model.fit(X_train, y_train)
None

In [199]:
#Проверка метрики
y_pred = model.predict(X_test)

print("test roc-auc:", roc_auc_score(y_test, y_pred))

test roc-auc: 0.6420604185004228


In [149]:
from sklearn.model_selection import GridSearchCV
#будем перебирать гиперпараметры для XGB 
n_estimators = np.linspace(50, 500, 100)
eta = [0.01, 0.05, 0.1, 0.2]


param_grid = [
    {'eta' : eta},
    {'n_estimators' : n_estimators}
  ]
xgb_model = XGBClassifier(random_state = 0, eval_metric='mlogloss', 
                          use_label_encoder=False,
                          n_jobs=-1)
grid_search = GridSearchCV(xgb_model, param_grid, cv=5,
                           scoring=auc_scorer,
                           return_train_score=True)

grid_search.fit(X_train, y_train)

None

C:\Grisha\Staj\conda\lib\site-packages\sklearn\model_selection\_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: 'numpy.float64' object cannot be interpreted as an integer

  warnings.warn("Estimator fit failed. The score on this train-test"


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False,
                                     eval_metric='mlogloss', gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None...
       413.63636364, 418.18181818, 422.72727273, 427.27272727,
       431.81818182, 436.36363636, 440.90909091, 445.45454545,
       450.        , 454.54545455, 459.09090909, 463.63636364,
       468.18181818, 472.72727273, 477.27272727, 481.81818182,
       486.36363636, 490.9090909

In [150]:
print("train score:", grid_search.best_score_)
print("eta:", grid_search.best_estimator_.get_params()["eta"], "\t", "n_estimators:",
    grid_search.best_estimator_.get_params()["n_estimators"])

y_pred = model.predict(X_test)
print("test roc-auc:", roc_auc_score(y_test, y_pred))

train score: 0.6489185887896615
eta: 0.1 	 n_estimators: 100
test roc-auc: 0.6441757969033239


In [686]:
#Предобработаем признаки перед тем как использовать нейронную сеть
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

cat_to_transform_cols = ['lobby_type', 'r1_hero', 'first_blood_player1', 'first_blood_player2',
                         'first_blood_team' 
                        ]
num_features = [feat for feat in features if feat not in cat_to_transform_cols]

#кодирование категориальных признаков
cat_data = pd.get_dummies(data[cat_to_transform_cols].applymap(lambda x: str(x)), drop_first=True)
data = pd.concat([data[num_features], cat_data], axis=1)

#разбиение на тест и трейн
train_data, test_data = train_test_split(data, test_size=0.2, random_state=0)

#нормализация численных признаков
for feat in num_features:
    scaler = StandardScaler()
    train_data[feat] = scaler.fit_transform(train_data[feat][:, np.newaxis]) 
    test_data[feat] = scaler.transform(test_data[feat][:, np.newaxis]).squeeze()

In [688]:
train_data.head(3)

,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,r2_xp,...,first_blood_player2_2.0,first_blood_player2_3.0,first_blood_player2_4.0,first_blood_player2_5.0,first_blood_player2_6.0,first_blood_player2_7.0,first_blood_player2_8.0,first_blood_player2_9.0,first_blood_team_0.0,first_blood_team_1.0
11153,-1.294497,-1.307383,-0.747402,-1.242827,-0.538007,-0.577933,0.692287,0.267122,-1.240466,-0.882647,...,0,0,0,0,0,0,0,0,1,0
20559,1.400398,1.399843,0.808301,0.638986,0.970857,-0.577933,0.292138,-0.772275,-0.328650,-0.325279,...,0,0,0,0,0,0,0,0,1,0
71813,-0.396199,-0.818529,-0.969645,-0.800047,-0.538007,-0.577933,0.292138,1.000813,0.583167,0.529713,...,1,0,0,0,0,0,0,0,0,1


In [210]:
!pip install torch

  Using cached torch-1.10.1-cp38-cp38-win_amd64.whl (226.6 MB)


In [212]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [689]:
#Создадим обьекты dataset и dataloader для загрузки наших данных
class dataset_(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        sample_x = self.X[idx, :]
        sample_y = self.y[idx]
        out_y = torch.zeros(len(np.unique(self.y)), dtype=torch.float)
        out_y[sample_y.item()] = 1
        #return {"sample" : torch.tensor(sample_x, dtype=torch.float), 
        #        "target" : torch.tensor(sample_y, dtype=torch.float)}   
        return {"sample" : torch.tensor(sample_x, dtype=torch.float), 
                "target" : out_y}   

In [700]:
X_train = train_data.to_numpy()
X_test = test_data.to_numpy()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2, random_state=0)

train_dataset = dataset_(X_train, y_train)
val_dataset = dataset_(X_val, y_val)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, 
                                           shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1000, shuffle=True)

In [771]:
#Инициализируем модель
class neural_model(nn.Module):
    def __init__(self, input_n):
        super(neural_model, self).__init__()
        self.f1 = nn.Linear(input_n, 256)
        self.f2 = nn.Linear(256, 412)
        self.f3 = nn.Linear(412, 256)
        self.f4 = nn.Linear(256, 2)
        self.out = nn.Softmax(dim=-1)
        
    def forward(self, x):
        x = F.relu(self.f1(x))
        x = F.relu(self.f2(x))
        x = F.relu(self.f3(x))
        x = self.f4(x)
        return self.out(x)


model = neural_model(X_train.shape[1])
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [773]:
#Опишем тренеровочный цикл
from tqdm.notebook import tqdm



epoches = 20
for epoch in tqdm(range(epoches)):
    for train_batch in train_loader:
        x_train_batch = train_batch["sample"]
        y_train_batch = train_batch["target"]
        
        optimizer.zero_grad()
        y_pred = model(x_train_batch)
        loss = nn.CrossEntropyLoss()(y_pred, y_train_batch)
        loss.backward()
        optimizer.step()
        
    if epoch % 2 == 0:
        mean_loss = []
        mean_val_acc = []
        mean_val_prec = []
        mean_val_recall = []
        mean_val_roc_auc = []
        with torch.no_grad():
            for val_batch in val_loader:
                x_val_batch = val_batch["sample"]
                y_val_batch = val_batch["target"]

                y_pred = model(x_val_batch)
                loss = nn.CrossEntropyLoss()(y_pred, y_val_batch)
                
                y_pred = [pred.argmax().item() for pred in y_pred]
                y_val_batch = [true.argmax().item() for true in y_val_batch]    
                mean_loss.append(loss.numpy())
                mean_val_acc.append(accuracy_score(y_val_batch, y_pred))
                mean_val_prec.append(precision_score(y_val_batch, y_pred))
                mean_val_recall.append(recall_score(y_val_batch, y_pred))
                mean_val_roc_auc.append(roc_auc_score(y_val_batch, y_pred))

            print("val loss:", sum(mean_loss) / len(mean_loss), 
                  "acc:", sum(mean_val_acc) / len(mean_val_acc), 
                  "prec:", sum(mean_val_prec) / len(mean_val_prec), 
                  "rec:", sum(mean_val_recall) / len(mean_val_recall), 
                  "roc:", sum(mean_val_roc_auc) / len(mean_val_roc_auc)
             )   

val loss: 0.6905851364135742 acc: 0.5223061041292639 prec: 0.521263138593472 rec: 0.998914334007633 roc: 0.5023595788720455
val loss: 0.6814878918230534 acc: 0.597794434470377 prec: 0.5709117806048699 rec: 0.9108538584137849 roc: 0.5851222916045254
val loss: 0.6465945355594158 acc: 0.6423850987432675 prec: 0.6479293302185021 rec: 0.6838242929673555 roc: 0.6404123781899738
val loss: 0.630992703139782 acc: 0.6475455565529623 prec: 0.6628284248409184 rec: 0.6581304749107877 roc: 0.6470933941080801
val loss: 0.6263057477772236 acc: 0.652241472172352 prec: 0.6623001220102082 rec: 0.6776260504343438 roc: 0.6511699251966507
val loss: 0.6252595894038677 acc: 0.6550100987432674 prec: 0.6653227633887088 rec: 0.6771880239841337 roc: 0.6541148269530535
val loss: 0.6241387240588665 acc: 0.6575667639138241 prec: 0.6695158624606421 rec: 0.6764904200891905 roc: 0.6567726574751576
val loss: 0.6244946867227554 acc: 0.6555327648114901 prec: 0.6716231277301686 rec: 0.6625341637512439 roc: 0.65530896942188

In [774]:
test_dataset = dataset_(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=X_test.shape[0])

In [776]:
#Проверка качества на тесте
with torch.no_grad():
    test_batch = next(iter(test_loader))
    x_test_batch = test_batch["sample"]
    y_test_batch = test_batch["target"]

    y_pred = model(x_test_batch)
    y_pred = [pred.argmax().item() for pred in y_pred]
    y_test_batch = [true.argmax().item() for true in y_test_batch] 

    print("test", "acc:", accuracy_score(y_test_batch, y_pred), 
          "prec:", precision_score(y_test_batch, y_pred), 
          "rec:", recall_score(y_test_batch, y_pred), 
          "roc:", roc_auc_score(y_test_batch, y_pred)
     )  

test acc: 0.6501594158181632 prec: 0.6641520525129813 rec: 0.6676186724443569 roc: 0.64934958589932


In [934]:
#Преобразуем данные для создания временного ряда
data = pd.read_csv('features.csv')

col_ind_nan = np.arange(data.columns.shape[0])[data.isnull().sum() > 0]
col_nan = data.columns[col_ind_nan]


event_time_features = ["first_blood_time", "radiant_bottle_time", "radiant_courier_time", 
                       "radiant_flying_courier_time", "radiant_first_ward_time", "dire_bottle_time", 
                      "dire_courier_time", "dire_flying_courier_time", "dire_first_ward_time"]

for feat in event_time_features:
    data[feat] = data[feat].fillna(-1)

data["first_blood_player1"] = data["first_blood_player1"].fillna(0)
data["first_blood_player2"] = data["first_blood_player2"].fillna(0)

data["first_blood_team"] = data["first_blood_team"].fillna(-1)

target = ['radiant_win']
features = list(filter(lambda x: x != target[0], data.columns))

added_features = ["duration",
                 "tower_status_radiant",
                 "tower_status_dire",
                 "barracks_status_radiant",
                 "barracks_status_dire"]

for feat in added_features: features.remove(feat)
    
features.remove("match_id")
features.remove("start_time")

features = np.array(features)

In [837]:
#Каждый обьект будет представлять собою 4 вектора x0, x1, x2, x3, x4
#xi содержит инфомацию о событиях произошедших между i и i + 1 минутами
min_time = -300 #начало игры
max_time = 300 #конец игры

diff = abs(max_time - min_time) // 4
time_range = [min_time + i * diff for i in range(5)]
X = np.array([[[1 if (time_range[j] < data[col][i] <= time_range[j + 1]) else 0 for col in event_time_features] for j in range(4)] for i in range(data.shape[0])])
y = data[target].to_numpy()

X.shape #количеств обьектов x 4 x длина вектора

In [935]:
not_events_cols = [col for col in features if col not in event_time_features]



cat_to_transform_cols = ['lobby_type', 'r1_hero', 'first_blood_player1', 'first_blood_player2',
                         'first_blood_team' 
                        ]
num_features = [feat for feat in not_events_cols if feat not in cat_to_transform_cols]

#кодирование категориальных признаков
cat_data = pd.get_dummies(data[cat_to_transform_cols].applymap(lambda x: str(x)), drop_first=True)
adding_data = pd.concat([data[num_features], cat_data], axis=1)

#разбиение на тест и трейн
train_adding, test_adding = train_test_split(adding_data, test_size=0.2, random_state=0)

#нормализация численных признаков
for feat in num_features:
    scaler = StandardScaler()
    train_adding[feat] = scaler.fit_transform(train_data[feat][:, np.newaxis]) 
    test_adding[feat] = scaler.transform(test_data[feat][:, np.newaxis]).squeeze()

In [873]:
#Создадим обьекты dataset и dataloader для загрузки наших данных
class time_dataset(torch.utils.data.Dataset):
    def __init__(self, X, y, adding_data):
        self.X = X
        self.y = y
        self.adding_data = adding_data
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        sample_x = self.X[idx, :, :]
        sample_add = self.adding_data[idx, :]
        sample_y = self.y[idx]
        out_y = torch.zeros(len(np.unique(self.y)), dtype=torch.float)
        out_y[sample_y.item()] = 1  
        return {"sample" : torch.tensor(sample_x, dtype=torch.float),
                "adding" : torch.tensor(sample_add, dtype=torch.float),
                "target" : out_y}

In [936]:
#разбиение на тренеровочну, валидационную, тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

train_adding, val_adding = train_test_split(train_adding, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2, random_state=0)

train_adding = train_adding.to_numpy()
val_adding = val_adding.to_numpy()
test_adding = test_adding.to_numpy()


train_dataset = time_dataset(X_train, y_train, train_adding)
val_dataset = time_dataset(X_val, y_val, val_adding)
test_dataset = time_dataset(X_test, y_test, test_adding)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, 
                                           shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1000, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=X_test.shape[0], shuffle=True)

In [944]:
#Изменим модель реккуретной нейронной сети.
#Так как мы имеем данные, которые не привязаны ко времени (после 1й минуты после 2й и тд.)
#но которые являются результатом после 5ти минут игры(количество смертей героя), то
#будем эти данные подавать после скрытых слоев на вход нейронной сети, чтобы они тоже
#вносили свой вклад в предсказание

class RNN(nn.Module):
    def __init__(self, input_size):
        super(RNN, self).__init__()
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=128, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(in_features=728, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=2)
        self.out = nn.Softmax(dim=-1)
        
        
    def forward(self, x, adding):
        x = torch.tanh(self.lstm1(x)[0])
        x = x.reshape(x.shape[0], -1)
        
        #добавляем данные после пяти минут игры
        x = torch.concat([x, adding], axis=1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return self.out(x)
    

In [945]:
input_size = X.shape[2]
model = RNN(input_size)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [946]:
train_batch = next(iter(train_loader))
x_train_batch = train_batch["sample"]
adding_batch = train_batch["adding"]
y_train_batch = train_batch["target"]

y_pred = model(x_train_batch, adding_batch)
loss = nn.CrossEntropyLoss()(y_pred, y_train_batch)

In [947]:
epoches = 20
for epoch in tqdm(range(epoches)):
    for train_batch in train_loader:
        x_train_batch = train_batch["sample"]
        train_adding_batch = train_batch["adding"]
        y_train_batch = train_batch["target"]
        
        optimizer.zero_grad()
        y_pred = model(x_train_batch, train_adding_batch)
        loss = nn.CrossEntropyLoss()(y_pred, y_train_batch)
        loss.backward()
        optimizer.step()
        
    if epoch % 2 == 0:
        mean_loss = []
        mean_val_acc = []
        mean_val_prec = []
        mean_val_recall = []
        mean_val_roc_auc = []
        with torch.no_grad():
            for val_batch in val_loader:
                x_val_batch = val_batch["sample"]
                val_adding_batch = val_batch["adding"]
                y_val_batch = val_batch["target"]

                y_pred = model(x_val_batch, val_adding_batch)
                loss = nn.CrossEntropyLoss()(y_pred, y_val_batch)
                
                y_pred = [pred.argmax().item() for pred in y_pred]
                y_val_batch = [true.argmax().item() for true in y_val_batch]    
                mean_loss.append(loss.numpy())
                mean_val_acc.append(accuracy_score(y_val_batch, y_pred))
                mean_val_prec.append(precision_score(y_val_batch, y_pred))
                mean_val_recall.append(recall_score(y_val_batch, y_pred))
                mean_val_roc_auc.append(roc_auc_score(y_val_batch, y_pred))

            print("val loss:", sum(mean_loss) / len(mean_loss), 
                  "acc:", sum(mean_val_acc) / len(mean_val_acc), 
                  "prec:", sum(mean_val_prec) / len(mean_val_prec), 
                  "rec:", sum(mean_val_recall) / len(mean_val_recall), 
                  "roc:", sum(mean_val_roc_auc) / len(mean_val_roc_auc)
             )   

val loss: 0.6879000812768936 acc: 0.5559238105924597 prec: 0.5410128062142929 rec: 0.9650511237897996 roc: 0.5388138099399032
val loss: 0.663640946149826 acc: 0.6342374326750448 prec: 0.6268816567615105 rec: 0.7361092217813624 roc: 0.6298759405385359
val loss: 0.6372556276619434 acc: 0.6450981822262118 prec: 0.6568846296036409 rec: 0.6684315746886387 roc: 0.6438658125921302
val loss: 0.6295940577983856 acc: 0.6529205565529624 prec: 0.6631092973602071 rec: 0.6751374923514516 roc: 0.6519274902170477
val loss: 0.6263429261744022 acc: 0.6533324730700178 prec: 0.6616667901606311 rec: 0.681781853553855 roc: 0.6520965663274095
val loss: 0.6252095587551594 acc: 0.6528368491921006 prec: 0.661391161288172 rec: 0.6843636950464209 roc: 0.6516872990307945
val loss: 0.6244991086423397 acc: 0.6546848070017954 prec: 0.6668060037885433 rec: 0.6728381071321846 roc: 0.6537923897571469
val loss: 0.6238073222339153 acc: 0.6548280969479354 prec: 0.6647388701371408 rec: 0.679923601678941 roc: 0.6537126458853

In [948]:
#Проверка качества на тесте
with torch.no_grad():
    test_batch = next(iter(test_loader))
    x_test_batch = test_batch["sample"]
    test_adding_batch = test_batch["adding"]
    y_test_batch = test_batch["target"]

    y_pred = model(x_test_batch, test_adding_batch)
    y_pred = [pred.argmax().item() for pred in y_pred]
    y_test_batch = [true.argmax().item() for true in y_test_batch] 

    print("test", "acc:", accuracy_score(y_test_batch, y_pred), 
          "prec:", precision_score(y_test_batch, y_pred), 
          "rec:", recall_score(y_test_batch, y_pred), 
          "roc:", roc_auc_score(y_test_batch, y_pred)
     )  

test acc: 0.6524220919469299 prec: 0.6653677545056016 rec: 0.6726413236163088 roc: 0.6514842433837033


In [ ]:
#Итого наилучшее качество 0.6514 дала реккурентная нейронная сеть c использованием 4х
#входных векторов, а так же добавлением к полносвязному слою вектор исходных данных.